## Data collection and manipulation

<p>Import Python libraries and rocket launch data</p>
You now have a goal: Is a launch likely to happen given specific weather conditions? You have a data set that contains weather data from:

- Several successful launches 
- One pushed launch day 
- The days leading up to and following each launch

Tools
- pandas jupyter seaborn scikit-learn keras tensorflow

Setup environment:
 - conda create -n devintrods python=3.10 pandas jupyter seaborn scikit-learn keras tensorflow
Activate environment
 - conda activate devintrods
 - pip install -U azureml-train-automl
VS Code tools
- Azure Account
- Azure Machine Learning
  

Visual Studio Code, Python, scikit-learn, and Azure.

In [2]:
import pandas as pd
import numpy as np

# # Sklearn library contains all the machine learning packages we need to digest and extract patterns from the data
from sklearn import linear_model, model_selection, metrics
from sklearn.model_selection import train_test_split

# Machine learning libraries used to build a decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# Sklearn's preprocessing library is used for processing and cleaning the data 
from sklearn import preprocessing

# for visualizing the tree
# import pydotplus
from IPython.display import Image 


Read data into a variable

In [3]:
launch_data = pd.read_excel('data/RocketLaunchDataCompleted.xlsx')
launch_data.head()

,Name,Date,Time (East Coast),Location,Crewed or Uncrewed,Launched?,High Temp,Low Temp,Ave Temp,Temp at Launch Time,...,Max Wind Speed,Visibility,Wind Speed at Launch Time,Hist Ave Max Wind Speed,Hist Ave Visibility,Sea Level Pressure,Hist Ave Sea Level Pressure,Day Length,Condition,Notes
0,NaN,1958-12-04,NaN,Cape Canaveral,NaN,NaN,75.0,68.0,71.00,NaN,...,16.0,15.0,NaN,NaN,NaN,30.22,NaN,10:26:00,Cloudy,NaN
1,NaN,1958-12-05,NaN,Cape Canaveral,NaN,NaN,78.0,70.0,73.39,NaN,...,14.0,10.0,NaN,NaN,NaN,30.2,NaN,10:26:00,Cloudy,NaN
2,Pioneer 3,1958-12-06,01:45:00,Cape Canaveral,Uncrewed,Y,73.0,0.0,60.21,62.0,...,15.0,10.0,11.0,NaN,NaN,30.25,NaN,10:25:00,Cloudy,NaN
3,NaN,1958-12-07,NaN,Cape Canaveral,NaN,NaN,76.0,57.0,66.04,NaN,...,10.0,10.0,NaN,NaN,NaN,30.28,NaN,10:25:00,Partly Cloudy,NaN
4,NaN,1958-12-08,NaN,Cape Canaveral,NaN,NaN,79.0,60.0,70.52,NaN,...,12.0,10.0,NaN,NaN,NaN,30.23,NaN,12:24:00,Partly Cloudy,NaN


Begin exploring data

In [6]:
launch_data.describe()

,High Temp,Low Temp,Ave Temp,Temp at Launch Time,Hist High Temp,Hist Low Temp,Hist Ave Temp,Percipitation at Launch Time,Hist Ave Percipitation,Max Wind Speed,Visibility,Wind Speed at Launch Time,Hist Ave Max Wind Speed,Hist Ave Visibility,Hist Ave Sea Level Pressure
count,299.000000,299.000000,299.000000,59.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,59.000000,0.0,0.0,0.0
mean,81.394649,38.745819,69.747124,75.101695,81.852843,62.872910,72.389900,0.063043,0.413478,16.842809,12.929766,10.593220,NaN,NaN,NaN
std,9.026700,33.423090,10.867407,10.471134,6.860432,8.806109,7.825282,0.211995,4.676693,4.701710,6.044445,4.672438,NaN,NaN,NaN
min,51.000000,0.000000,29.040000,50.000000,71.000000,49.000000,60.000000,0.000000,0.060000,8.000000,7.000000,2.000000,NaN,NaN,NaN
25%,77.000000,0.000000,63.050000,70.000000,75.000000,55.000000,65.000000,0.000000,0.080000,14.000000,10.000000,7.000000,NaN,NaN,NaN
50%,82.000000,51.000000,71.610000,77.000000,82.000000,64.000000,72.000000,0.000000,0.110000,16.000000,10.000000,10.000000,NaN,NaN,NaN
75%,88.000000,72.000000,78.530000,81.500000,88.000000,73.000000,80.000000,0.000000,0.200000,18.000000,15.000000,12.500000,NaN,NaN,NaN
max,99.000000,83.000000,90.790000,98.000000,91.000000,79.000000,82.000000,1.800000,81.000000,60.000000,80.000000,26.000000,NaN,NaN,NaN


In [7]:
launch_data.columns

Index(['Name', 'Date', 'Time (East Coast)', 'Location', 'Crewed or Uncrewed',
       'Launched?', 'High Temp', 'Low Temp', 'Ave Temp', 'Temp at Launch Time',
       'Hist High Temp', 'Hist Low Temp', 'Hist Ave Temp',
       'Percipitation at Launch Time', 'Hist Ave Percipitation',
       'Wind Direction', 'Max Wind Speed', 'Visibility',
       'Wind Speed at Launch Time', 'Hist Ave Max Wind Speed',
       'Hist Ave Visibility', 'Sea Level Pressure',
       'Hist Ave Sea Level Pressure', 'Day Length', 'Condition', 'Notes'],
      dtype='object')

Exercise - Clean weather data to analyze rocket launch criteria

## Data cleaning
The first step in cleaning your data is to replace all missing values with something

In [8]:
# Get overview of data
launch_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Name                          60 non-null     object        
 1   Date                          300 non-null    datetime64[ns]
 2   Time (East Coast)             59 non-null     object        
 3   Location                      300 non-null    object        
 4   Crewed or Uncrewed            60 non-null     object        
 5   Launched?                     60 non-null     object        
 6   High Temp                     299 non-null    float64       
 7   Low Temp                      299 non-null    float64       
 8   Ave Temp                      299 non-null    float64       
 9   Temp at Launch Time           59 non-null     float64       
 10  Hist High Temp                299 non-null    float64       
 11  Hist Low Temp                 29

### Here are a few ways we'll clean the data:

- The rows that don't have Y in the Launched column didn't have a rocket launch, so make those missing values N.
- For rows missing information on whether the rocket was crewed or uncrewed, assume uncrewed. Uncrewed is more likely because there were fewer crewed missions. 
- For missing wind direction, mark it as unknown.
- For missing condition data, assume it was a typical day and use fair.
- For any other data, use a value of 0.

In [16]:
launch_data['Launched?'].fillna('N', inplace=True)
launch_data['Crewed or Uncrewed'].fillna('Uncrewed', inplace=True)
launch_data['Wind Direction'].fillna('unknown', inplace=True)
launch_data['Condition'].fillna('Fair',inplace=True)
launch_data.fillna(0,inplace=True)
launch_data.head()


,Name,Date,Time (East Coast),Location,Crewed or Uncrewed,Launched?,High Temp,Low Temp,Ave Temp,Temp at Launch Time,...,Max Wind Speed,Visibility,Wind Speed at Launch Time,Hist Ave Max Wind Speed,Hist Ave Visibility,Sea Level Pressure,Hist Ave Sea Level Pressure,Day Length,Condition,Notes
0,0,1958-12-04,0,Cape Canaveral,Uncrewed,N,75.0,68.0,71.00,0.0,...,16.0,15.0,0.0,0.0,0.0,30.22,0.0,10:26:00,Cloudy,0
1,0,1958-12-05,0,Cape Canaveral,Uncrewed,N,78.0,70.0,73.39,0.0,...,14.0,10.0,0.0,0.0,0.0,30.2,0.0,10:26:00,Cloudy,0
2,Pioneer 3,1958-12-06,01:45:00,Cape Canaveral,Uncrewed,Y,73.0,0.0,60.21,62.0,...,15.0,10.0,11.0,0.0,0.0,30.25,0.0,10:25:00,Cloudy,0
3,0,1958-12-07,0,Cape Canaveral,Uncrewed,N,76.0,57.0,66.04,0.0,...,10.0,10.0,0.0,0.0,0.0,30.28,0.0,10:25:00,Partly Cloudy,0
4,0,1958-12-08,0,Cape Canaveral,Uncrewed,N,79.0,60.0,70.52,0.0,...,12.0,10.0,0.0,0.0,0.0,30.23,0.0,12:24:00,Partly Cloudy,0


In [18]:
launch_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Name                          300 non-null    object        
 1   Date                          300 non-null    datetime64[ns]
 2   Time (East Coast)             300 non-null    object        
 3   Location                      300 non-null    object        
 4   Crewed or Uncrewed            300 non-null    object        
 5   Launched?                     300 non-null    object        
 6   High Temp                     300 non-null    float64       
 7   Low Temp                      300 non-null    float64       
 8   Ave Temp                      300 non-null    float64       
 9   Temp at Launch Time           300 non-null    float64       
 10  Hist High Temp                300 non-null    float64       
 11  Hist Low Temp                 30

### Data Manipulation

In [20]:
# As part of the data cleaning process, we have to convert text data to 
# numerical because computers understand only numbers
label_encoder = preprocessing.LabelEncoder()

# Three columns have categorical text info, and we convert them to numbers
launch_data['Crewed or Uncrewed']=label_encoder.fit_transform(launch_data['Crewed or Uncrewed'])
launch_data['Wind Direction'] = label_encoder.fit_transform(launch_data['Wind Direction'])
launch_data['Condition'] = label_encoder.fit_transform(launch_data['Condition'])

In [22]:
launch_data.head(20)

,Name,Date,Time (East Coast),Location,Crewed or Uncrewed,Launched?,High Temp,Low Temp,Ave Temp,Temp at Launch Time,...,Max Wind Speed,Visibility,Wind Speed at Launch Time,Hist Ave Max Wind Speed,Hist Ave Visibility,Sea Level Pressure,Hist Ave Sea Level Pressure,Day Length,Condition,Notes
0,0,1958-12-04,0,Cape Canaveral,1,N,75.0,68.0,71.00,0.0,...,16.0,15.0,0.0,0.0,0.0,30.22,0.0,10:26:00,0,0
1,0,1958-12-05,0,Cape Canaveral,1,N,78.0,70.0,73.39,0.0,...,14.0,10.0,0.0,0.0,0.0,30.2,0.0,10:26:00,0,0
2,Pioneer 3,1958-12-06,01:45:00,Cape Canaveral,1,Y,73.0,0.0,60.21,62.0,...,15.0,10.0,11.0,0.0,0.0,30.25,0.0,10:25:00,0,0
3,0,1958-12-07,0,Cape Canaveral,1,N,76.0,57.0,66.04,0.0,...,10.0,10.0,0.0,0.0,0.0,30.28,0.0,10:25:00,6,0
4,0,1958-12-08,0,Cape Canaveral,1,N,79.0,60.0,70.52,0.0,...,12.0,10.0,0.0,0.0,0.0,30.23,0.0,12:24:00,6,0
5,0,1959-03-01,0,Cape Canaveral,1,N,78.0,0.0,60.92,0.0,...,10.0,10.0,0.0,0.0,0.0,30.13,0.0,11:35:00,0,0
6,0,1959-03-02,0,Cape Canaveral,1,N,82.0,0.0,65.12,0.0,...,13.0,10.0,0.0,0.0,0.0,30.19,0.0,11:37:00,6,0
7,Pioneer 4,1959-03-03,13:10:00,Cape Canaveral,1,Y,78.0,0.0,69.65,78.0,...,20.0,7.0,12.0,0.0,0.0,30.13,0.0,11:38:00,0,0
8,0,1959-03-04,0,Cape Canaveral,1,N,77.0,0.0,66.72,0.0,...,21.0,7.0,0.0,0.0,0.0,30.11,0.0,11:40:00,0,0
9,0,1959-03-05,0,Cape Canaveral,1,N,81.0,67.0,72.63,0.0,...,17.0,10.0,0.0,0.0,0.0,30.12,0.0,11:42:00,0,0
